In [1]:
import pandas as pd

from utils.common import day_aware_shift, plot_confusion_matrix, read_bm_data
from utils.features import create_lag, create_rsi, create_dst, create_ma_ratio, create_z, \
    create_bollinger_band, create_money_flow_index, create_macd_diff, \
    create_ppo, create_pvo

import pickle
import datetime
import ta

In [2]:
# df_wti_1m = pickle.load(open('./large_files/cl-1m-2.pkl', 'rb'))
# df_wti_1m = df_wti_1m[(df_wti_1m['datetime'].dt.time >= datetime.time(hour=7, minute=0)) & (df_wti_1m['datetime'].dt.hour <= 16)]

df_1m = read_bm_data('data/btm/gc-1m.csv')
df_1m = df_1m[(df_1m['datetime'].dt.time >= datetime.time(hour=7, minute=0)) & (df_1m['datetime'].dt.hour <= 16)]

df = df_1m.copy()
df['month'] = df['datetime'].dt.month
df.head(3)

,date,time,open,high,low,close,volume,datetime,day,month
380,2007-04-02,0700,669.4,669.4,669.4,669.4,1,2007-04-02 07:00:00-04:00,Monday,4
381,2007-04-02,0701,669.4,669.4,669.3,669.3,2,2007-04-02 07:01:00-04:00,Monday,4
382,2007-04-02,0702,669.2,669.3,669.2,669.2,5,2007-04-02 07:02:00-04:00,Monday,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2546127 entries, 380 to 5782315
Data columns (total 10 columns):
 #   Column    Dtype                           
---  ------    -----                           
 0   date      object                          
 1   time      object                          
 2   open      float64                         
 3   high      float64                         
 4   low       float64                         
 5   close     float64                         
 6   volume    int64                           
 7   datetime  datetime64[ns, America/New_York]
 8   day       object                          
 9   month     int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(2), object(3)
memory usage: 213.7+ MB


In [6]:
with pd.HDFStore('./large_files/yahoo.h5') as store:
    df = store.get('data/CL=F')
df['datetime'] = df.index
df = df[(df['datetime'].dt.time >= datetime.time(hour=7, minute=0)) & (df['datetime'].dt.hour <= 16)]
df['month'] = df['datetime'].dt.month
df['date'] = df['datetime'].dt.strftime('%Y-%m-%d')
df['time'] = df['datetime'].dt.strftime('%H%M')
df['day'] = df['datetime'].dt.strftime('%A')
df = df.reset_index(drop=True)
df.head(3)

,open,high,low,close,adj close,volume,datetime,month,date,time,day
0,76.629997,76.690002,76.629997,76.68,76.68,462,2024-02-21 07:00:00-05:00,2,2024-02-21,0700,Wednesday
1,76.680000,76.690002,76.660004,76.68,76.68,141,2024-02-21 07:01:00-05:00,2,2024-02-21,0701,Wednesday
2,76.690002,76.709999,76.680000,76.68,76.68,164,2024-02-21 07:02:00-05:00,2,2024-02-21,0702,Wednesday


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11337 entries, 0 to 11336
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype                           
---  ------     --------------  -----                           
 0   open       11337 non-null  float64                         
 1   high       11337 non-null  float64                         
 2   low        11337 non-null  float64                         
 3   close      11337 non-null  float64                         
 4   adj close  11337 non-null  float64                         
 5   volume     11337 non-null  int64                           
 6   datetime   11337 non-null  datetime64[ns, America/New_York]
 7   month      11337 non-null  int64                           
 8   date       11337 non-null  object                          
 9   time       11337 non-null  object                          
 10  day        11337 non-null  object                          
dtypes: datetime64[ns, America/New_York](1), f

# Impute missing minutes

In [8]:
df_result = []
removed_day = []
for day, df_day in df.groupby(pd.Grouper(key='datetime', freq='D')):
    if df_day.shape[0] == 0:
        continue
    df_day = df_day.copy()
    df_day['datetime'] = pd.to_datetime(df_day['datetime'])

    df_trade_hour = df_day[(df_day['datetime'].dt.hour >= 9) & (df_day['datetime'].dt.hour < 16)]

    # allow up to 1 hour data missing in main trading hours
    if df_trade_hour.shape[0] < 360:
        print(f'{day} {df_trade_hour.shape[0]}')
        removed_day.append(day)
        continue

    year = day.year
    month = day.month
    day_of_month = day.day
    start_time = pd.Timestamp(year=year, month=month, day=day_of_month, hour=9, tz='America/New_York')
    end_time = pd.Timestamp(year=year, month=month, day=day_of_month, hour=15, minute=59, tz='America/New_York')
    desired_index = pd.to_datetime(pd.date_range(start=start_time, end=end_time, freq='T')).tolist()

    df_tmp = pd.DataFrame(desired_index, columns=['datetime'])
    df_trade_hour = pd.merge(df_trade_hour, df_tmp, how='outer', on='datetime').sort_values('datetime')

    df_trade_hour['date'] = df_trade_hour['datetime'].dt.strftime('%Y-%m-%d')
    df_trade_hour['time'] = df_trade_hour['datetime'].dt.strftime('%H%M')
    df_trade_hour['day'] = df_trade_hour['datetime'].dt.strftime('%A')
    df_trade_hour['month'] = df_trade_hour['datetime'].dt.month
    df_trade_hour[['open', 'high', 'low', 'close', 'volume']] = df_trade_hour[['open', 'high', 'low', 'close', 'volume']].interpolate(axis=0).round(2)
    df_trade_hour[['open', 'high', 'low', 'close', 'volume']] = df_trade_hour[['open', 'high', 'low', 'close', 'volume']].bfill(axis=0)
    df_trade_hour['volume'] = df_trade_hour['volume'].astype(int)
    
    df_day = pd.concat([df_day, df_trade_hour]).drop_duplicates(subset='datetime').sort_values('datetime')

    df_result.append(df_day)

df2 = pd.concat(df_result)
df3 = df2[(df2['datetime'].dt.hour >= 9) & (df2['datetime'].dt.hour < 16)]

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11381 entries, 0 to 11336
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype                           
---  ------     --------------  -----                           
 0   open       11381 non-null  float64                         
 1   high       11381 non-null  float64                         
 2   low        11381 non-null  float64                         
 3   close      11381 non-null  float64                         
 4   adj close  11337 non-null  float64                         
 5   volume     11381 non-null  int64                           
 6   datetime   11381 non-null  datetime64[ns, America/New_York]
 7   month      11381 non-null  int64                           
 8   date       11381 non-null  object                          
 9   time       11381 non-null  object                          
 10  day        11381 non-null  object                          
dtypes: datetime64[ns, America/New_York](1), f

In [10]:
with pd.HDFStore('./large_files/yahoo.h5') as store:
    store.put('data/cl', df2, format='table')
    # store.put('data/data_imputed_trade_hour_20240226', df3, format='table')
    print(list(store.keys()))

['/data/BZ=F', '/data/CL=F', '/data/GC=F', '/data/cl']


In [5]:
with pd.HDFStore('./large_files/data2.h5') as store:
    df2 = store.get('data/data_imputed_20240217')
    df3 = store.get('data/data_imputed_trade_hour_20240217')
    print(list(store.keys()))

['/data/data_imputed_20240217', '/data/data_imputed_trade_hour_20240217', '/data/features_20240215', '/data/features_20240215/meta/values_block_4/meta', '/data/features_20240215/meta/values_block_3/meta', '/data/features_20240215/meta/values_block_2/meta', '/data/features_20240215/meta/values_block_1/meta', '/data/features_20240215/meta/values_block_0/meta']


# Technical Indicators

In [3]:
df = create_bollinger_band(df, period=30)
df = create_bollinger_band(df, period=15)
df = create_bollinger_band(df, period=10)
df['dollar_vol'] = df[['close', 'volume']].prod(axis=1)
df.info(verbose=3, show_counts=True)

bollinger_band_10: 100%|██████████| 6158/6158 [00:04<00:00, 1486.31it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568270 entries, 525 to 116265
Data columns (total 17 columns):
 #   Column      Non-Null Count    Dtype                           
---  ------      --------------    -----                           
 0   date        2568270 non-null  object                          
 1   time        2568270 non-null  object                          
 2   open        2568270 non-null  float64                         
 3   high        2568270 non-null  float64                         
 4   low         2568270 non-null  float64                         
 5   close       2568270 non-null  float64                         
 6   volume      2568270 non-null  int64                           
 7   datetime    2568270 non-null  datetime64[ns, America/New_York]
 8   day         2568270 non-null  object                          
 9   month       2568270 non-null  int64                           
 10  bb_30_high  2442062 non-null  float64                         
 1

In [4]:
for period in [3, 5, 10, 15, 20, 30, 60]:
    df = create_z(df, period, 'close')
    df = create_z(df, period, 'volume')
    df = create_z(df, period, 'dollar_vol')

df.info(verbose=3, show_counts=True)

z_60 dollar_vol: 100%|██████████| 6158/6158 [00:04<00:00, 1393.38it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568270 entries, 525 to 116265
Data columns (total 101 columns):
 #    Column              Non-Null Count    Dtype                           
---   ------              --------------    -----                           
 0    date                2568270 non-null  object                          
 1    time                2568270 non-null  object                          
 2    open                2568270 non-null  float64                         
 3    high                2568270 non-null  float64                         
 4    low                 2568270 non-null  float64                         
 5    close               2568270 non-null  float64                         
 6    volume              2568270 non-null  int64                           
 7    datetime            2568270 non-null  datetime64[ns, America/New_York]
 8    day                 2568270 non-null  object                          
 9    month               2568270 non-

In [5]:
df = create_dst(df, [3, 5, 10, 15, 30, 60])
df = create_dst(df, [3, 5, 10, 15, 30, 60], 'dollar_vol')
df = create_rsi(df, [5, 10, 15, 30, 60])

df.info(verbose=3, show_counts=True)

rsi: 100%|██████████| 6158/6158 [00:34<00:00, 178.88it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568270 entries, 525 to 116265
Data columns (total 171 columns):
 #    Column                        Non-Null Count    Dtype                           
---   ------                        --------------    -----                           
 0    date                          2568270 non-null  object                          
 1    time                          2568270 non-null  object                          
 2    open                          2568270 non-null  float64                         
 3    high                          2568270 non-null  float64                         
 4    low                           2568270 non-null  float64                         
 5    close                         2568270 non-null  float64                         
 6    volume                        2568270 non-null  int64                           
 7    datetime                      2568270 non-null  datetime64[ns, America/New_York]
 8    day      

In [6]:
df = create_money_flow_index(df, 5)
df = create_money_flow_index(df, 15)
df = create_money_flow_index(df, 30)

df = create_macd_diff(df, 5, 15, 60)
df = create_macd_diff(df, 5, 30, 60)
df = create_macd_diff(df, 3, 10, 30)

df = create_ppo(df, 5, 15, 60)
df = create_ppo(df, 3, 10, 30)
df = create_pvo(df, 5, 15, 60)
df = create_pvo(df, 3, 10, 30)

pvo30_3_10: 100%|██████████| 6158/6158 [00:03<00:00, 1923.94it/s]


In [7]:
for period in [5, 15, 30, 60]:
    for sub_period in [5, 15, 30, 60]:
        if sub_period > period:
                df = create_ma_ratio(df, period, sub_period)

df.info(verbose=3, show_counts=True)

ma_ratio_30_60: 100%|██████████| 6158/6158 [00:02<00:00, 2645.85it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568270 entries, 525 to 116265
Data columns (total 187 columns):
 #    Column                        Non-Null Count    Dtype                           
---   ------                        --------------    -----                           
 0    date                          2568270 non-null  object                          
 1    time                          2568270 non-null  object                          
 2    open                          2568270 non-null  float64                         
 3    high                          2568270 non-null  float64                         
 4    low                           2568270 non-null  float64                         
 5    close                         2568270 non-null  float64                         
 6    volume                        2568270 non-null  int64                           
 7    datetime                      2568270 non-null  datetime64[ns, America/New_York]
 8    day      

In [9]:
for period in [3, 5, 10, 15, 30, 60]:
    for sub_period in [3, 5, 10, 15, 30, 60]:
        # df = create_lag(df, f'z_close_{period}m', sub_period)
        # df = create_lag(df, f'z_volume_{period}m', sub_period)
        # df = create_lag(df, f'dst_close_high_{period}m', sub_period)
        # df = create_lag(df, f'dst_close_low_{period}m', sub_period)
        df = create_lag(df, f'dst_dollar_vol_high_{period}m', sub_period)
        df = create_lag(df, f'dst_dollar_vol_low_{period}m', sub_period)

df.info(verbose=3, show_counts=True)

lag60m_dst_dollar_vol_low_60m: 100%|██████████| 6158/6158 [00:01<00:00, 3833.87it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568270 entries, 525 to 116265
Data columns (total 259 columns):
 #    Column                          Non-Null Count    Dtype                           
---   ------                          --------------    -----                           
 0    date                            2568270 non-null  object                          
 1    time                            2568270 non-null  object                          
 2    open                            2568270 non-null  float64                         
 3    high                            2568270 non-null  float64                         
 4    low                             2568270 non-null  float64                         
 5    close                           2568270 non-null  float64                         
 6    volume                          2568270 non-null  int64                           
 7    datetime                        2568270 non-null  datetime64[ns, America/New_Y

In [10]:
with pd.HDFStore('./large_files/data2.h5') as store:
    store.put('data/features_20240215', df, format='table')

In [11]:
with pd.HDFStore('./large_files/data2.h5') as store:
    print(list(store.keys()))

['/data/features_20240215', '/data/features_20240215/meta/values_block_4/meta', '/data/features_20240215/meta/values_block_3/meta', '/data/features_20240215/meta/values_block_2/meta', '/data/features_20240215/meta/values_block_1/meta', '/data/features_20240215/meta/values_block_0/meta']
